In [25]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.externals import joblib
from tsr_hog_extractor import HOGExtractor
import random as rand
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from helpers import print_executed
from helpers import plot_learning_curve
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd
import datetime
import warnings

#########################################################################
#                                                                       #
# If you use features from project files you can ommit code below       #
#                                                                       #
#########################################################################

hog_parameters = {
    'color_model': 'hsv',  # hls, hsv, yuv, ycrcb
    'svc_input_size': 64,  
    'number_of_orientations': 11,  # 6 - 12
    'pixels_per_cell': 16,  # 8, 16
    'cells_per_block': 2,  # 1, 2
    'do_transform_sqrt': True,
}

extractor = HOGExtractor(hog_parameters)
positive_images, negative_images = [], []
    
print_executed()



 EXECUTED BLOCK AT 1:34:41


In [26]:

# read file which describe positive dataset
positive_data_descr = pd.read_csv('dataset/positive/folders_classes.csv', delimiter=';', dtype=object)
positive_data_descr = positive_data_descr.drop('opis', axis=1)

# select only prohibitory signs which are speed limits
positive_data_descr = \
    positive_data_descr.loc[(positive_data_descr['typ'] == 'a') (positive_data_descr['numer'] == '33')]
print(positive_data_descr, '\n')

# read file which describe types of signs and select its language
sign_description = pd.read_csv('dataset/sign_types.csv', delimiter=';')
sign_description = sign_description.drop('opis', axis=1)
print(sign_description.head(5), '\n')
print_executed()


   folder typ numer
11  00011   a    6a
13  00013   a     7
18  00018   a    30
19  00019   a     1
20  00020   a     2
21  00021   a     4
22  00022   a    11
23  00023   a    15
24  00024   a   12b
25  00025   a    14
26  00026   a    29
27  00027   a    16
28  00028   a    17
29  00029   a    24
30  00030   a    32
31  00031   a   18b 

  typ    description
0   a        warning
1   b    prohibitory
2   c      mandatory
3   d  informational 


 EXECUTED BLOCK AT 1:34:51


In [3]:
print("Loading negative images to memory...")

# this parameter determines if use a generator from GTSDB or manually collected images
useGenerator = True
if useGenerator:
    from model_negative_data_generator import NegativeDataGenerator
    ndg = NegativeDataGenerator()
    negative_images = ndg()
else:
    negative_paths = glob.glob('./dataset/negative/*/*.png')
    for path in tqdm(negative_paths):
        image = cv2.imread(path)
        if image.shape[0] >= 30 or image.shape[1] >= 30:
            negative_images.append(image)

print_executed()


/Users/michal/PycharmProjects/HOG_TSR/object_detector.py:20: UserWarning: SVC path or scaler path not passed! Used default classifier.
  warnings.warn("SVC path or scaler path not passed! Used default classifier.")
155it [00:49,  2.70it/s]


Loading negative images to memory...
We will slice 155 negative images!
It can take a while ...
Extracted 66960 slices from negative images.

 EXECUTED BLOCK AT 23:15:48


In [27]:
print("Loading positive images to memory...\n")

# for each selected folder
for folder in positive_data_descr['folder']:
    # read all images from it and sort it
    paths = sorted(glob.glob('./dataset/positive/' + folder + '/*.ppm'))
    # read csv which contains bounding boxes of traffic sign
    csv = pd.read_csv('./dataset/positive/' + folder + '/GT-' + folder + '.csv', delimiter=';')
    print("\nReading folder: ", folder, '\n')
    for path in tqdm(paths):
        # read name of the image
        name = path.split('/')[4]
        # search for selected image in the csv and flatten that row to list
        row = (csv.loc[csv['Filename'] == name]).values.tolist()
        # read image and cut ROI with traffic sign, then add to list
        image = cv2.imread(path)
        if image.shape[0] >= 35 or image.shape[1] >= 35:
            image = image[row[0][3]:row[0][5], row[0][4]:row[0][6]]
            #print(image.shape)
            positive_images.append(image)

print_executed()


100%|██████████| 780/780 [00:02<00:00, 332.60it/s]


Loading positive images to memory...


Reading folder:  00011 


Reading folder:  00013 


Reading folder:  00018 


Reading folder:  00019 


Reading folder:  00020 


Reading folder:  00021 


Reading folder:  00022 


Reading folder:  00023 


Reading folder:  00024 


Reading folder:  00025 


Reading folder:  00026 


Reading folder:  00027 


Reading folder:  00028 


Reading folder:  00029 


Reading folder:  00030 


Reading folder:  00031 


 EXECUTED BLOCK AT 1:36:1


In [29]:
# stack already read images to np array
# shapes of arrays can be different, because we have stack images with different shapes to one array
positive_images, negative_images = np.asarray(positive_images), np.asarray(negative_images)
total_signs, total_nonsigns = positive_images.shape[0], negative_images.shape[0]

print("Positive images shape: ", len(positive_images), "type: ", type(positive_images))
print("Negative images shape: ", len(negative_images), "type: ", type(negative_images))
print("Total positive signs: ", total_signs, "\nTotal nonsigns: ", total_nonsigns)

positive_features, negative_features = [], []
print_executed()


Positive images shape:  9893 type:  <class 'numpy.ndarray'>
Negative images shape:  1 type:  <class 'numpy.ndarray'>
Total positive signs:  9893 
Total nonsigns:  0

 EXECUTED BLOCK AT 1:37:49


In [30]:
print("Extracting features from traffic signs...")

for img in tqdm(positive_images):
    positive_features.append(extractor.features(img)) 

print("Extracting features from non traffic signs...")
for img in tqdm(negative_images):
    negative_features.append(extractor.features(img)) 

positive_features = np.asarray(positive_features)
negative_features = np.asarray(negative_features)

# You can safe your features to file
# np.save('trained_models/negative_features', negative_features)
# np.save('trained_models/positive_features', positive_features)

print_executed()


  0%|          | 0/1 [00:00<?, ?it/s]


Extracting features from traffic signs...
Extracting features from non traffic signs...


error: OpenCV(3.4.2) /opt/concourse/worker/volumes/live/9523d527-1b9e-48e0-7ed0-a36adde286f0/volume/opencv-suite_1535558719691/work/modules/imgproc/src/color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'CvtHelper'


In [18]:
#########################################################################
#                                                                       #
# If you use features from project files you can start here             #
#                                                                       #
#########################################################################

positive_features = np.load('trained_models/positive_features.npy')
negative_features = np.load('trained_models/negative_features.npy')

positive_features = np.asarray(positive_features)
negative_features = np.asarray(negative_features)

# operation to balance a size of negative features
negative_features = negative_features[:positive_features.shape[0]]

total_signs = positive_features.shape[0]
total_nonsigns = negative_features.shape[0]

print_executed()


 EXECUTED BLOCK AT 0:58:51


In [19]:
print("Scaling features...")
print(len(positive_features), len(positive_features[0]))
print(len(negative_features), len(negative_features[0]))

unscaled_x = np.vstack((positive_features, negative_features)).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
y = np.hstack((np.ones(total_signs), np.zeros(total_nonsigns)))

print(" x shape: ", x.shape, " y shape: ", y.shape)

print_executed()


Scaling features...
9183 1188
9183 1188
 x shape:  (18366, 1188)  y shape:  (18366,)

 EXECUTED BLOCK AT 0:59:14


In [15]:
print("Plot learning curve for dataset")

svc = SVC(probability=True)
title = "Learning curves for Supported Vector Machine"
warnings.filterwarnings("ignore")
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
plot_learning_curve(svc, title, x, y, cv=cv, ylim=(0.7, 1.01))

print_executed()


Plot learning curve for dataset



 EXECUTED BLOCK AT 18:7:37


In [20]:
print("Training classifier and saving models...")

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=rand.randint(1, 100))

svc = SVC(probability=True)
warnings.filterwarnings("ignore")
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)
y_pred_proba = svc.predict_proba(x_test)
y_pred = np.argmax(y_pred_proba, axis=1)
print(classification_report(y_test, y_pred))
print("overall accuracy:", np.round(accuracy, 4))
print(confusion_matrix(y_test, y_pred))
print(y_pred_proba)


# save models
now = datetime.datetime.now()
created = str(now.year)+str(now.month)+str(now.day)  # +str(now.hour)+str(now.minute)
joblib.dump(svc, './trained_models/SVC_'+created+'.pkl')
joblib.dump(scaler, './trained_models/scaler_'+created+'.pkl')


print_executed()


Training classifier and saving models
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1826
         1.0       1.00      1.00      1.00      1848

   micro avg       1.00      1.00      1.00      3674
   macro avg       1.00      1.00      1.00      3674
weighted avg       1.00      1.00      1.00      3674

overall accuracy: 0.9992
[[1826    0]
 [   2 1846]]
[[3.00000090e-14 1.00000000e+00]
 [9.99976825e-01 2.31754257e-05]
 [9.99412950e-01 5.87050409e-04]
 ...
 [3.00000090e-14 1.00000000e+00]
 [3.17807850e-07 9.99999682e-01]
 [1.36973522e-12 1.00000000e+00]]

 EXECUTED BLOCK AT 1:1:37


In [14]:
print("Test prediction of single ROI")

f1 = extractor.features(negative_images[rand.randint(0, len(negative_images))])
f2 = extractor.features(positive_images[rand.randint(0, len(positive_images))])
f3 = extractor.features(positive_images[rand.randint(0, len(positive_images))])
f4 = extractor.features(negative_images[rand.randint(0, len(negative_images))])

fA = scaler.transform([f1, f2, f3, f4])
print(svc.predict(fA))


f5 = extractor.features(negative_images[rand.randint(2000, total_nonsigns)])
f6 = extractor.features(positive_images[rand.randint(2000, total_signs)])
f7 = extractor.features(positive_images[rand.randint(2000, total_signs)])
f8 = extractor.features(negative_images[rand.randint(2000, total_nonsigns)])

fB = scaler.transform([f5, f6, f7, f8])
print(svc.predict_proba(fB))



Test prediction of single ROI


IndexError: list index out of range